In [16]:
import time
import logging
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def obtener_lista_encuestas_dropdown():
    """
    Usa un enfoque semiautomático para que el usuario prepare la página.
    Luego, el script hace clic en el dropdown de 'Select2', espera a que
    las opciones aparezcan y las extrae a un DataFrame.
    """
    url = "https://proyectos.inei.gob.pe/microdatos/"
    print(f"Abriendo el navegador en: {url}")

    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # --- PAUSA PARA INTERVENCIÓN MANUAL ---
        input(
            "\n" + "="*50 +
            "\nPAUSA: El navegador está bajo tu control." +
            "\n   1. Cierra el pop-up de anuncio." +
            "\n   2. Haz clic en la pestaña 'Consulta por Encuestas'." +
            "\n\nUna vez que veas el dropdown '(Elija una Encuesta)', regresa aquí y presiona 'Enter'..." +
            "\n" + "="*50 + "\n"
        )
        
        logging.info("Reanudando el script. Extrayendo datos del dropdown...")

        # --- EXTRACCIÓN DE OPCIONES DEL DROPDOWN ---
        # 1. Hacer clic en el dropdown para abrirlo
        # Este es el elemento que tú encontraste.
        dropdown_trigger = wait.until(
            EC.element_to_be_clickable((By.ID, "select2-cmbEncuesta0ID-container"))
        )
        dropdown_trigger.click()
        logging.info("Dropdown de encuestas abierto.")
        
        # 2. Esperar a que las opciones aparezcan y obtenerlas
        # Las opciones de Select2 suelen aparecer en una lista <ul> con esta clase
        options_xpath = "//ul[contains(@class, 'select2-results__options')]/li"
        wait.until(EC.presence_of_element_located((By.XPATH, options_xpath)))
        
        # Leemos todos los elementos <li> de la lista de resultados
        option_elements = driver.find_elements(By.XPATH, options_xpath)
        
        nombres_encuestas = []
        for option in option_elements:
            # Extraemos el texto de cada opción
            texto_opcion = option.text
            # Ignoramos las opciones vacías o que son placeholders
            if texto_opcion and texto_opcion != "(Elija una Encuesta)":
                nombres_encuestas.append(texto_opcion)

        logging.info(f"Se extrajeron {len(nombres_encuestas)} nombres de encuestas del dropdown.")
        
        if not nombres_encuestas:
            logging.error("No se encontraron nombres de encuestas en el dropdown.")
            return None
        
        # 3. Crear el DataFrame final
        # Nota: Desde el dropdown solo podemos obtener el nombre. Las otras columnas no están disponibles aquí.
        df = pd.DataFrame(nombres_encuestas, columns=['Nombre_Encuesta'])
        df['Entidad_Ejecutora'] = 'No disponible desde el dropdown'
        df['Anio_Publicacion'] = 'No disponible desde el dropdown'

        logging.info(f"¡Éxito! Se creó el DataFrame con {len(df)} encuestas.")
        return df

    except Exception as e:
        logging.error("Ocurrió un error durante la ejecución del script.", exc_info=True)
        return None
    finally:
        logging.info("Proceso finalizado. Cerrando el navegador.")
        driver.quit()

# --- Ejecución ---
lista_de_encuestas = obtener_lista_encuestas_dropdown()

if lista_de_encuestas is not None:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    print("\n--- LISTA DE ENCUESTAS DISPONIBLES EN EL DROPDOWN DEL INEI ---")
    print(lista_de_encuestas)

2025-09-30 11:41:46,613 - INFO - ====== WebDriver manager ======


Abriendo el navegador en: https://proyectos.inei.gob.pe/microdatos/


2025-09-30 11:41:51,028 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 11:41:51,344 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 11:41:51,582 - INFO - Driver [C:\Users\Alvaro\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-30 11:42:03,301 - INFO - Reanudando el script. Extrayendo datos del dropdown...
2025-09-30 11:42:03,564 - INFO - Dropdown de encuestas abierto.
2025-09-30 11:42:04,822 - INFO - Se extrajeron 53 nombres de encuestas del dropdown.
2025-09-30 11:42:04,854 - INFO - ¡Éxito! Se creó el DataFrame con 53 encuestas.
2025-09-30 11:42:04,856 - INFO - Proceso finalizado. Cerrando el navegador.



--- LISTA DE ENCUESTAS DISPONIBLES EN EL DROPDOWN DEL INEI ---
                                                                                        Nombre_Encuesta  \
0                                                                            ENAHO Metodología ANTERIOR   
1                                                                         ENAHO Metodología ACTUALIZADA   
2                                                         ENCUESTA PERMANENTE DE EMPLEO NACIONAL - EPEN   
3                                                                    CENSO DE INFRAESTRUCTURA EDUCATIVA   
4                                                                 CENSO NACIONAL AGROPECUARIO - CENAGRO   
5                           CENSO NACIONAL AGROPECUARIO - CENAGRO - HOJAS DE INFORMACIÓN COMPLEMENTARIA   
6                                                                          CENSO NACIONAL DE COMISARIAS   
7                                                        CENSO NACIONAL DE MERCA

In [ ]:
# Lista COMPLETA de encuestas de interés que vamos a intentar descargar.
# El script iterará sobre esta lista, seleccionando cada una en el dropdown.
target_encuestas = [
    # --- Prioridad Alta ---
    "ENAHO Metodología ACTUALIZADA",
    "ENCUESTA DEMOGRÁFICA Y DE SALUD FAMILIAR - ENDES",
    "MAPA DE POBREZA",
    "ENCUESTA NACIONAL DE PROGRAMAS PRESUPUESTALES - ENAPRES",
    
    # --- Prioridad Secundaria ---
    "REGISTRO NACIONAL DE MUNICIPALIDADES - RENAMU",
    "CENSO NACIONAL AGROPECUARIO - CENAGRO",
    "CENSO NACIONAL DE MERCADOS DE ABASTOS - CENAMA",
    "ENCUESTA NACIONAL ESPECIALIZADA SOBRE DISCAPACIDAD - ENEDIS",
    "ENAHO Metodología ANTERIOR" 
]

print(f"Se intentará descargar un total de {len(target_encuestas)} encuestas:")
for i, encuesta in enumerate(target_encuestas):
    print(f"{i+1}. {encuesta}")
    

Se intentará descargar un total de 9 encuestas:
1. ENAHO Metodología ACTUALIZADA
2. ENCUESTA DEMOGRÁFICA Y DE SALUD FAMILIAR - ENDES
3. MAPA DE POBREZA
4. ENCUESTA NACIONAL DE PROGRAMAS PRESUPUESTALES - ENAPRES
5. REGISTRO NACIONAL DE MUNICIPALIDADES - RENAMU
6. CENSO NACIONAL AGROPECUARIO - CENAGRO
7. CENSO NACIONAL DE MERCADOS DE ABASTOS - CENAMA
8. ENCUESTA NACIONAL ESPECIALIZADA SOBRE DISCAPACIDAD - ENEDIS
9. ENAHO Metodología ANTERIOR


In [25]:
import os
import time
import logging
import pandas as pd
import zipfile
import shutil
import re
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ==============================================================================
# --- CONFIGURACIÓN ---
# ==============================================================================
BASE_SAVE_DIR = r'D:\UNIDAD D\UNIVERSIDAD\2025-2\Modelos Fisiológicos\Modelos_fisio\datasets\INEI_Datasets'
FORMATO_PREFERIDO = 'STATA' # Se buscará el enlace con este título o texto.

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# --- FUNCIONES DE AYUDA (Adaptadas a la nueva lógica) ---

def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name).replace(' ', '_').strip()

def get_options_from_select(select_element):
    options = [opt.get_attribute('text') for opt in select_element.find_elements(By.TAG_NAME, 'option')]
    return [opt for opt in options if opt and not opt.startswith('<') and not opt.startswith('(') and opt.strip() != '-']

def wait_for_spinner(wait):
    wait.until(EC.invisibility_of_element_located((By.ID, "div_espera")))
    time.sleep(1) # Pausa de estabilidad extra

def process_single_download(temp_dir, final_path_info):
    try:
        start_time, timeout, zip_path = time.time(), 600, None
        while time.time() - start_time < timeout:
            zips = list(Path(temp_dir).glob('*.zip'))
            if zips and not list(Path(temp_dir).glob('*.crdownload')):
                time.sleep(5); zip_path = zips[0]; break
            time.sleep(1)
        if not zip_path: logging.warning("   -> Timeout en la descarga."); return
        
        extract_folder = Path(final_path_info['dir']) / zip_path.stem
        final_csv_path = Path(final_path_info['dir']) / (final_path_info['name'] + ".csv")
        os.makedirs(final_path_info['dir'], exist_ok=True)
        
        with zipfile.ZipFile(zip_path, 'r') as zf: zf.extractall(extract_folder)
        
        data_file = next(extract_folder.glob('**/*.dta'), None) or next(extract_folder.glob('**/*.sav'), None)
        if data_file:
            df = pd.read_stata(data_file) if data_file.suffix == '.dta' else pd.read_spss(data_file)
            df.to_csv(final_csv_path, index=False)
            logging.info(f"   -> ÉXITO: Guardado como {final_csv_path.name}")
        else: logging.warning("   -> No se encontró archivo .dta o .sav en el zip.")
    except Exception as e: logging.error(f"   -> Fallo en proceso de archivo: {e}")
    finally:
        if 'zip_path' in locals() and zip_path and zip_path.exists(): zip_path.unlink()
        if 'extract_folder' in locals() and extract_folder and extract_folder.exists(): shutil.rmtree(extract_folder)

# --- SCRIPT PRINCIPAL ---

def main_downloader_enaho_focused():
    TEMP_DOWNLOAD_DIR = Path(BASE_SAVE_DIR) / 'temp_downloads'
    os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)
    
    opts = webdriver.ChromeOptions()
    prefs = {"download.default_directory": str(TEMP_DOWNLOAD_DIR)}; opts.add_experimental_option("prefs", prefs)
    opts.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    
    try:
        driver.get("https://proyectos.inei.gob.pe/microdatos/")
        wait = WebDriverWait(driver, 60)
        input("\n" + "="*50 + "\nPAUSA: Prepara el navegador.\n   1. Cierra el pop-up.\n   2. Clic en 'Consulta por Encuestas'.\n\nPresiona 'Enter' para continuar...\n" + "="*50 + "\n")
        
        logging.info("--- INICIANDO PROCESO PARA: ENAHO Metodología ACTUALIZADA ---")
        
        # 1. Seleccionar la encuesta principal
        wait.until(EC.element_to_be_clickable((By.ID, "select2-cmbEncuesta0ID-container"))).click()
        option_xpath = "//ul[contains(@class, 'select2-results__options')]//li[text()='ENAHO Metodología ACTUALIZADA']"
        wait.until(EC.element_to_be_clickable((By.XPATH, option_xpath))).click(); wait_for_spinner(wait)

        # 2. Seleccionar Sub-Encuesta
        sub_survey_xpath = "//div[@id='ENAHON']//select"
        wait.until(EC.presence_of_element_located((By.XPATH, f"{sub_survey_xpath}/option[2]")))
        sub_survey_element = driver.find_element(By.XPATH, sub_survey_xpath)
        sub_surveys = get_options_from_select(sub_survey_element)
        print(f"\nSub-Encuestas encontradas: {sub_surveys}")
        sub_survey_to_select = sub_surveys[0] # Elegimos la primera por ahora
        logging.info(f"Seleccionando: '{sub_survey_to_select}'")
        Select(sub_survey_element).select_by_visible_text(sub_survey_to_select); wait_for_spinner(wait)
        
        # 3. Seleccionar Año
        year_xpath = "//select[@name='cmbAnno']"
        wait.until(EC.presence_of_element_located((By.XPATH, f"{year_xpath}/option[2]")))
        year_element = driver.find_element(By.XPATH, year_xpath)
        years = get_options_from_select(year_element)
        print(f"\nAños encontrados: {years}")
        year_to_select = years[-1] # Elegimos el último (más reciente) por ahora
        logging.info(f"Seleccionando año: '{year_to_select}'")
        Select(year_element).select_by_visible_text(year_to_select); wait_for_spinner(wait)
        
        # 4. Seleccionar Período
        period_xpath = "//div[@id='divPeriodo']/select"
        wait.until(EC.presence_of_element_located((By.XPATH, f"{period_xpath}/option[2]")))
        period_element = driver.find_element(By.XPATH, period_xpath)
        periods = get_options_from_select(period_element)
        print(f"\nPeríodos encontrados: {periods}")
        # Buscamos 'Anual', si no, elegimos el primero de la lista
        period_to_select = next((p for p in periods if 'Anual' in p), periods[0])
        logging.info(f"Seleccionando período: '{period_to_select}'")
        Select(period_element).select_by_visible_text(period_to_select); wait_for_spinner(wait)
        
        # 5. LA LÓGICA CLAVE: Procesar la tabla de módulos
        logging.info("\n--- Buscando tabla de módulos para descargar ---")
        wait.until(EC.visibility_of_element_located((By.ID, "divDetalle")))
        # El XPATH busca las filas <tr> que están dentro de la tabla anidada
        module_rows = driver.find_elements(By.XPATH, "//div[@id='divDetalle']//table/tbody/tr/td/table/tbody/tr")

        # La primera fila es el encabezado, la saltamos
        if len(module_rows) < 2:
            logging.warning("No se encontraron módulos en la tabla.")
            return

        for row in module_rows[1:]:
            try:
                cells = row.find_elements(By.TAG_NAME, 'td')
                if not cells or len(cells) < 7: continue

                module_name = cells[6].text.strip()
                logging.info(f"Procesando Módulo: '{module_name}'")
                
                # Buscar el enlace de descarga STATA por su título
                download_link = row.find_element(By.XPATH, f".//a[@title='Archivo {FORMATO_PREFERIDO}']")
                
                # Limpiar directorio temporal para esta descarga
                for f in Path(TEMP_DOWNLOAD_DIR).glob('*'): f.unlink()
                
                driver.execute_script("arguments[0].click();", download_link)
                
                # Definir nombres y rutas para esta descarga específica
                final_folder = Path(BASE_SAVE_DIR)/'ENAHO_Metodologia_ACTUALIZADA'/sanitize_filename(sub_survey_to_select)/year_to_select/sanitize_filename(period_to_select)
                final_filename = f"ENAHO_{year_to_select}_{sanitize_filename(period_to_select)}_{sanitize_filename(module_name)}"
                path_info = {'dir': final_folder, 'name': final_filename}
                
                process_single_download(TEMP_DOWNLOAD_DIR, path_info)

            except Exception as e:
                logging.error(f"  -> Fallo al procesar una fila de módulo. Error: {e}")
                continue

    except Exception as e:
        logging.error("Ocurrió un error fatal en el script.", exc_info=True)
    finally:
        input("Proceso terminado. Presiona Enter para cerrar el navegador.")
        driver.quit()
        if TEMP_DOWNLOAD_DIR.exists(): shutil.rmtree(TEMP_DOWNLOAD_DIR)
        logging.info("--- PROCESO FINALIZADO ---")

if __name__ == "__main__":
    main_downloader_enaho_focused()

2025-09-30 14:47:34,797 - INFO - ====== WebDriver manager ======
2025-09-30 14:47:35,838 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 14:47:35,989 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 14:47:36,140 - INFO - Driver [C:\Users\Alvaro\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-30 14:47:44,134 - INFO - --- INICIANDO PROCESO PARA: ENAHO Metodología ACTUALIZADA ---
2025-09-30 14:47:45,382 - INFO - Seleccionando: 'Condiciones de Vida y Pobreza - ENAHO'



Sub-Encuestas encontradas: ['Condiciones de Vida y Pobreza - ENAHO', 'Condiciones de vida y pobreza - ENAHO PANEL']


2025-09-30 14:47:46,669 - INFO - Seleccionando año: '2025'



Años encontrados: ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']


2025-09-30 14:47:47,822 - INFO - Seleccionando período: 'Trimestre 1'



Períodos encontrados: ['Trimestre 1', 'Trimestre 2']


2025-09-30 14:47:48,954 - INFO - 
--- Buscando tabla de módulos para descargar ---
2025-09-30 14:47:49,022 - INFO - Procesando Módulo: 'Características de la Vivienda y del Hogar'
2025-09-30 14:47:58,171 - INFO -    -> ÉXITO: Guardado como ENAHO_2025_Trimestre_1_Características_de_la_Vivienda_y_del_Hogar.csv
2025-09-30 14:47:58,198 - INFO - Procesando Módulo: 'Características de los Miembros del Hogar'
2025-09-30 14:49:05,614 - INFO -    -> ÉXITO: Guardado como ENAHO_2025_Trimestre_1_Características_de_los_Miembros_del_Hogar.csv
2025-09-30 14:49:05,633 - INFO - Procesando Módulo: 'Educación'
2025-09-30 14:49:12,161 - ERROR -    -> Fallo en proceso de archivo: 
Value labels for column p301a1 are not unique. These cannot be converted to
pandas categoricals.

Either read the file with `convert_categoricals` set to False or use the
low level interface in `StataReader` to separately read the values and the
value_labels.

The repeated labels are:
---------------------------------------------